In [1]:
import json
import pandas as pd

In [2]:
def extract_nodes(plan_node, depth=0, parent=None, path='root'):
    """Рекурсивный обход узлов плана"""
    row = {
        'Node Type': plan_node.get('Node Type'),
        'Relation Name': plan_node.get('Relation Name', ''),
        'Actual Total Time': plan_node.get('Actual Total Time'),
        'Actual Rows': plan_node.get('Actual Rows'),
        'Depth': depth,
        'Parent': parent,
        'Path': path,
    }
    rows = [row]
    
    for i, subplan in enumerate(plan_node.get('Plans', [])):
        subpath = f"{path}.{i}"
        rows += extract_nodes(subplan, depth + 1, plan_node.get('Node Type'), subpath)
    
    return rows

In [3]:
with open("explain.json") as f:
    explain_data = json.load(f)

In [4]:
root_plan = explain_data[0]['Plan']
nodes = extract_nodes(root_plan)

In [6]:
df = pd.DataFrame(nodes)
df

,Node Type,Relation Name,Actual Total Time,Actual Rows,Depth,Parent,Path
0,Hash Join,,0.065,1,0,None,root
1,Seq Scan,employees,0.034,5,1,Hash Join,root.0
2,Hash,,0.018,1,1,Hash Join,root.1
3,Seq Scan,departments,0.012,1,2,Hash,root.1.0
